Útskýringar eru í Kernel bible 2 ef þig þyrstir í aukaefni

<br>
Hér kemur bara eitthvað random dæmi, en á að virkar fyrir

$$f_{\theta}: \mathbb{R}^{n} \rightarrow \mathbb{R}^{m}$$

In [5]:
include("NN.jl")

model = Chain(Dense(784 => 2, sigmoid), Dense(2 => 10)) # W_2[1x2](W_1[2x1]x[1]+b_1[2x1])+b_2[1]

x1 = load_MNIST()[1][:,1];
x2 = load_MNIST()[1][:,2];

In [6]:
function Df(model, x)
    # x: single datapoint
    m = length(model(x))
    k = sum(length, Flux.params(model)) # Total amount of params

    # Þetta anonymous function reiknar gradient fyrir hvert function í outputinu frá 1:m
    jac = (fi) -> Flux.jacobian(() -> model(x)[fi],Flux.params(model)) # anonymous function

    # Skilgreini tómt Jacobian fylki
    Jacob = zeros(k,m)

    for func_i = 1:m
        current_col = []
        for param_i = 1:length(Flux.params(model))
            push!(current_col, jac(func_i)[Flux.params(model)[param_i]]) # Fyrir hvern parametra W1, B1, W2...
        end
        current_col = collect(Iterators.flatten(current_col)) # Flatten, flet allt
        # --- Spurning hvort hægt sé að gera þetta skilvirkara?
        # --- Held samt að Iterators pakkinn eigi að vera nokkuð skilvirkur

        Jacob[:, func_i] .= current_col # geri current_col að næsta dálka vigri jacobian
    end

    return Jacob # Þetta er Df fylkið í bilblíunni
end

function kernel(model, x)
    # x: dataset
    N = length(x)
    m = length(model(x[1]))  # Number of functions in the model output
    K = zeros(N*m, N*m)

    for i = 1:N
        for j = 1:N
            block = Df(model, x[i])' * Df(model, x[j])
            K[(i-1)*m+1:i*m, (j-1)*m+1:j*m] .= block
        end
    end

    return K
end

kernel (generic function with 2 methods)

In [8]:
x = [x1,x2]

K = kernel(model,x);
println("Kernel completed")

Kernel completed


In [9]:
using LinearAlgebra

# Check if K is ok
if K == K'
    println( "K is symmetric")
end

val = eigen(K).values
negVal = findall(x-> x < 0, val)

K is symmetric


Int64[]

Hér fyrir neðan er test, þar sem K er þekkt og var reiknað í Kernel bible 1

$$K = \begin{bmatrix}
    8 & 12 \\ 12 & 19
\end{bmatrix}$$

In [10]:
# TESTING

using Flux

model = Chain(Dense(1 => 2), Dense(2 => 1)) # W_2[1x2](W_1[2x1]x[1]+b_1[2x1])+b_2[1]

x1 = [1.0]
x2 = [2.0]

# Skilgreini parametra
W1 = Flux.params(model)[1]  # W_1
b1 = Flux.params(model)[2]  # b_1
W2 = Flux.params(model)[3]  # W_1
b2 = Flux.params(model)[4]  # b_1

# Breyti gildum í parametrum
W1 .= ones(2,1)  #  Hér má setja eitthvað "fixed" fylki, breyti gildum í W1
b1 .= [0;1]

W2 .= [1 0]
b2 .= 0

K = kernel(model, [x1,x2])


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(1 => 2)
│   summary(x) = 1-element Vector{Float64}
└ @ Flux C:\Users\axelb\.julia\packages\Flux\UsEXa\src\layers\stateless.jl:60


2×2 Matrix{Float64}:
  8.0  12.0
 12.0  19.0